**Code aimed at extracting the risks by plurals of selected purposes across three different focus groups.**

In [ ]:
!pip install plurals

In [ ]:
from plurals.helpers import print_anes_mapping
print_anes_mapping()

In [ ]:
# set environments variables
import os
os.environ["OPENAI_API_KEY"] = input("Enter your OpenAI API key: ")

In [4]:
from plurals.agent import Agent
from plurals.deliberation import Moderator, Ensemble

In [1]:
# Agent task
focus_group_task = (
    "You are a participant in a focus group. From your perspective, "
    "What are the possible risks of facial recognition technology with the purpose 'Identifying citizens for personalized services' ? "
    "Respond concisely (max 3 sentences), focusing on how this might affect you based on you gender, race, age and level of education."

)
MODEL="gpt-4o-mini"

id_use=88
purpose="Identifying citizens for personalized services"

**FIRST FOCUS GROUP: 100 agents with personas randomly sampled from the ANES**

In [27]:
# Create 100 agents with personas randomly sampled from the ANES dataset (the default dataset used by Plurals)
group_1_agents = [Agent(persona="random", model=MODEL, task=focus_group_task) for _ in range(100)]

In [28]:
# If you want to see the exact profiles of the personas
#for i, agent in enumerate(group_1_agents):
#    print(f"Agent {i+1}:\n{agent.persona}\n")

**SECOND FOUS GROUP: 100 agentS, black and woman, every other characteristics are random**

In [29]:
# Focus group composed of 100 Black women
group_2_agents = [
    Agent(
        query_str="gender4=='Woman'&race4=='Black'",
        model=MODEL,
        task=focus_group_task
    )
    for _ in range(100)
]

In [30]:
# If you want to see the exact profiles of the personas
#for i, agent in enumerate(group_2_agents):
#    print(f"Agent {i+1}:\n{agent.persona}\n")

**THIRD FOCUS GROUP: 100 agents with at least two marginalized characteristics, distributed proportionally to the population with the same characteristics in the ANES dataset**

In [31]:
groups = [

    {"query": "gender4 != 'Man'  and race4 != 'White'  and age < 65   and educ != 'No HS credential'",    "count": 30},
    {"query": "gender4 != 'Man'  and race4 == 'White'  and age >= 65  and educ != 'No HS credential'",    "count": 46},
    {"query": "gender4 != 'Man'  and educ == 'No HS credential'  and race4 == 'White'    and age < 65",            "count": 3},
    {"query": "gender4 != 'Man'  and race4 != 'White'  and age >= 65           and educ == 'No HS credential'",    "count": 1},
    {"query": "gender4 != 'Man'  and race4 != 'White'  and age >= 65           and educ != 'No HS credential'",    "count": 7},
    {"query": "gender4 != 'Man'  and race4 != 'White'  and educ == 'No HS credential'    and age < 65",            "count": 3},
    {"query": "gender4 != 'Man'  and age >= 65         and educ == 'No HS credential'    and race4 == 'White'",    "count": 2},
    {"query": "gender4 == 'Man'  and race4 != 'White'   and age >= 65          and educ != 'No HS credential'",    "count": 5},
    {"query": "gender4 == 'Man'  and race4 != 'White'   and educ == 'No HS credential' and age < 65",   "count": 2},
    {"query": "gender4 == 'Man'  and race4 != 'White'   and age >= 65     and educ == 'No HS credential' ",   "count": 1},

]


group_3_agents = []

for group in groups:
    for _ in range(group["count"]):
        group_3_agents.append(
            Agent(
                query_str=group["query"],
                model=MODEL,
                task=focus_group_task
            )
        )

**MODERATOR** (They are all the same. It's enough to define it just once now.)


In [32]:
from plurals.deliberation import Moderator

moderator = Moderator(
    model=MODEL,
    task=(
        "You are the moderator of the focus group. Your goal is to summarize the most relevant risks that participants raised in response to a specific AI use case."
    ),
    combination_instructions=(
      "Using ${previous_responses}, extract and list the most relevant and distinct risks mentioned by participants.\n"
      "- Only include risks that represent clearly different concerns.\n"
      "- Group together and rephrase any similar or overlapping risks into a single, representative statement.\n"
      "- Do not include risks that are minor variations or examples of the same broader concern.\n"
      "- Eliminate redundancy: do not list risks that are too close in meaning or implication.\n"
      "- Prioritize the most frequently mentioned or broadly impactful risks.\n"
      "Return a concise list with each risk as a separate, short sentence.\n"
      "Use this structure:\n"
      "1. [First distinct risk]\n"
      "2. [Second distinct risk]\n"
      "...\n\n"
      "Examples (format only — do not copy content):\n"
      "1. Undermines the right to privacy if used to track individuals without consent.\n"
      "2. Reinforces inequality if access to the technology is limited to certain groups.\n"
      "3. Leads to job losses if automation replaces human roles."
)
)


**Run of the three groups**

In [33]:
ensemble_1 = Ensemble(agents=group_1_agents, moderator=moderator)
ensemble_2 = Ensemble(agents=group_2_agents, moderator=moderator)
ensemble_3 = Ensemble(agents=group_3_agents, moderator=moderator)


focus_groups = [
    (1, ensemble_1, moderator),
    (2, ensemble_2, moderator),
    (3, ensemble_3, moderator)
]

In [34]:
ensemble_1.process()
ensemble_2.process()
ensemble_3.process()


'1. Leads to privacy violations due to tracking individuals without their consent.\n2. Creates increased surveillance culture, making individuals feel constantly monitored.\n3. Risks misidentification, particularly for marginalized groups based on race or age.\n4. Causes discriminatory outcomes in accessing services due to biased algorithms.\n5. Results in potential misuse of personal data, raising concerns about data privacy.\n6. Leaves vulnerable individuals without understanding of their rights or technology implications.\n7. Increases the likelihood of unfair profiling based on demographic characteristics.\n8. Contributes to a sense of distrust towards authorities and technological systems.\n9. Risks overlooking the needs of less tech-savvy populations in service provision.\n10. May establish an environment where personal freedoms are compromised.'

**Create one file json for each pourpose**

In [35]:
import json

use_infos = [
    {"id_group": 1 , "id_use": id_use, "purpose": purpose, "description": "General group"},
    {"id_group": 2 , "id_use": id_use, "purpose": purpose, "description": "All agents are black woman"},
    {"id_group": 3 , "id_use": id_use, "purpose": purpose, "description": "Agents rapresent the distribution of oppressed population"}
]


ensembles = [ensemble_1, ensemble_2, ensemble_3]

# Final structure
output = []

for ensemble, info in zip(ensembles, use_infos):
    risks = [line.strip() for line in ensemble.final_response.split("\n") if line.strip()]
    output.append({
        "id_use": info["id_use"],
        "purpose": info["purpose"],
        "id_group": info["id_group"],
        "description": info["description"],
        "Risks": risks
    })

# Change the name of the file
path = "88_Identifying citizens for personalized services.json"
with open(path, "w") as f:
    json.dump(output, f, indent=4)
path

'88_Identifying citizens for personalized services.json'

 **From this point on, the cells below are only for debugging purpose. If everything works, you can stop here. To debug, only the relevant cells should be run at the appropriate time**


In [17]:
ensemble_1 = Ensemble(agents=group_1_agents, moderator=moderator)

ensemble_1.process()
print(ensemble_1.final_response)



1. Could lead to misidentification or false positives, resulting in wrongful access denials or accusations against individuals.
2. Raises significant privacy concerns as personal data may be collected, stored, or misused without consent.
3. Contains inherent biases in algorithms, disproportionately affecting women, people of color, and older individuals.
4. Creates a sense of constant surveillance, which can make individuals feel uncomfortable and insecure in their daily lives.
5. Could disproportionately disadvantage marginalized groups in secure access situations, reinforcing systemic inequalities.
6. May lead to a lack of transparency about how data is used and the algorithms' decision-making processes, creating distrust.


In [42]:
ensemble_2 = Ensemble(agents=group_2_agents, moderator=moderator)

ensemble_2.process()
print(ensemble_2.final_response)



1. Facial recognition technology might misidentify individuals, particularly people of color, due to racial biases.
2. There is a risk of unfair treatment or access denial caused by misidentification.
3. Gender bias may cause inaccuracies in recognizing women.
4. Privacy concerns arise from personal data potentially being stored or used without consent.
5. Misuse or mishandling of collected personal data raises security concerns.
6. Older individuals may find it challenging to navigate or trust the technology.
7. Limited understanding due to lack of formal education can hinder effective data privacy management.
8. The transparent explanation of data collection and usage is insufficient.


In [ ]:
ensemble_3 = Ensemble(agents=group_3_agents, moderator=moderator)

ensemble_3.process()
print(ensemble_3.final_response)



In [ ]:
# Esegui la deliberazione
# ensemble.process()
# già inclusa nella funzione successiva


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

Error fetching response from LLM: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-7l732g5zvXUR96YL0uvAt3Gg on tokens per min (TPM): Limit 10000, Used 9774, Requested 247. Please try again in 125ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Error fetching response from LLM: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-7l732g5zvXUR96YL0uvAt3Gg on tokens per min (TPM): Limit 10000, Used 9773, Requested 248. Please try again in 125ms. Visit http

In [31]:
print(ensemble_1.final_response)

1. Leads to misidentification, especially for older individuals and those from diverse racial backgrounds, resulting in unfair access denials and scrutiny.  
2. Invades privacy by potentially tracking individuals without consent and misusing personal data.  
3. Exhibits bias against certain demographics, which could reinforce societal inequalities and result in discriminatory practices.  
4. Causes a lack of transparency about how data is collected, stored, and utilized, leaving individuals vulnerable to misuse.  
5. Creates a sense of constant surveillance, making people feel uncomfortable and less secure in public spaces.  
6. May not function accurately for individuals who do not fit typical profiles, leading to added complications for those with varied features or backgrounds.


In [32]:
print(ensemble_2.final_response)

1. Leads to misidentification, particularly for older individuals and people of color, resulting in unfair treatment or denied access.
2. Raises significant privacy concerns, as personal data could be collected, stored, and misused without consent.
3. Creates an environment of increased surveillance that makes individuals feel constantly monitored.
4. Reinforces bias in security systems, potentially discriminating against marginalized groups based on race, gender, or age.
5. Complicates access to services or spaces for those who do not fit the tech's typical demographic profiles, leading to exclusion.
6. Negatively impacts trust in security measures due to perceived unreliability and errors in the technology. 
7. Leaves individuals feeling vulnerable and unsure about their rights or how to protect their personal information in relation to this technology.


In [33]:
print(ensemble_3.final_response)

1. Leads to privacy invasion if personal data is monitored or collected without consent.
2. Risks misidentification, particularly affecting individuals from certain racial or age demographics, leading to unfair treatment.
3. Creates potential for discrimination against specific groups, including women and people of color, due to biases in algorithm accuracy.
4. Causes barriers to access when the technology fails, resulting in wrongful denial of entry to secure locations.
5. Raises concerns about surveillance overreach, making individuals feel under constant observation.
6. Risks misuse of collected data, leading to further privacy violations or identity theft.
7. Lacks transparency in its usage, generating distrust among users regarding how data is handled.
8. May create a chilling effect on personal freedom and expression due to the fear of being constantly monitored or misidentified.
